<a href="https://colab.research.google.com/github/Taehee-K/AI-Cycle/blob/master/result/sr_od_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Mount with your Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Setup darknet environment
(need CUDA 10.0 for darknet for colab)

### Uninstall & Reinstall CUDA

In [ ]:
#Uninstall the current CUDA version
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'nvidia-kernel-common-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-325-updates' for glob 'nvidia*'
Note, selecting 'nvidia-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-binary' for glob 'nvidia*'
Note, selecting 'nvidia-331-dev' for glob 'nvidia*'
Note, selecting 'nvidia-304-updates-dev' for glob 'nvidia*'
Note, selecting 'nvidia-compute-utils-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-384-dev' for glob 'nvidia*'
Note, selecting 'nvidia-libopencl1-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-340-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-dkms-450-server' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-source-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-331-updates-uvm' for glob 'nvidi

In [ ]:
#Download CUDA 10.0
!wget  --no-clobber https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-repo-ubuntu1804_10.0.130-1_amd64.deb
#install CUDA kit dpkg
!dpkg -i cuda-repo-ubuntu1804_10.0.130-1_amd64.deb
!sudo apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub
!apt-get update
!apt-get install cuda-10-0

--2021-06-20 11:20:33--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-repo-ubuntu1804_10.0.130-1_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.195.19.142
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2940 (2.9K) [application/x-deb]
Saving to: ‘cuda-repo-ubuntu1804_10.0.130-1_amd64.deb’

cuda-repo-ubuntu180 100%[===================>]   2.87K  --.-KB/s    in 0s      

2021-06-20 11:20:33 (155 MB/s) - ‘cuda-repo-ubuntu1804_10.0.130-1_amd64.deb’ saved [2940/2940]

Selecting previously unselected package cuda-repo-ubuntu1804.
(Reading database ... 123844 files and directories currently installed.)
Preparing to unpack cuda-repo-ubuntu1804_10.0.130-1_amd64.deb ...
Unpacking cuda-repo-ubuntu1804 (10.0.130-1) ...
Setting up cuda-repo-ubuntu1804 (10.0.130-1) ...

Configuration file '/etc/apt/sources.li

In [ ]:
# check whether GPU is provided
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


### Download Darknet

In [ ]:
import os
assert os.getcwd()=='/content', 'Directory should be "/content" instead of "{}"'.format(os.getcwd())

# remove the existing folder if have
#!rm -r darknet_for_colab

# download and compile darknet_for_colab
!git clone https://github.com/quangnhat185/darknet_for_colab.git
%cd darknet_for_colab
!make
!chmod +x ./darknet.py

Cloning into 'darknet_for_colab'...
remote: Enumerating objects: 1083, done.
remote: Total 1083 (delta 0), reused 0 (delta 0), pack-reused 1083
Receiving objects: 100% (1083/1083), 5.16 MiB | 22.60 MiB/s, done.
Resolving deltas: 100% (233/233), done.
/content/darknet_for_colab
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -fPIC -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In funct

In [ ]:
# install opencv
!pip install opencv-python

## Import Libraries

In [ ]:
from PIL import Image
import tensorflow as tf
import tarfile
import datetime
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.python.keras.layers import Conv2D, Input, Lambda, Activation
from tensorflow.python.keras.models import Model
from os.path import exists, join, basename
from os import makedirs, remove, path
from six.moves import urllib
from tensorflow import keras 

In [ ]:
SCALE = 2
# LR_SIZE = 100
# Make sure HR_SIZE is smaller than 300 since some images in the dataset have height/width < 300
# HR_SIZE = LR_SIZE * SCALE
IMG_SIZE = 300
HR_SIZE = IMG_SIZE * SCALE

BATCH_SIZE = 100

## Super Resolution

In [ ]:
def load_test_image(img_path):
    hr = img_path
    hr = cv2.cvtColor(hr, cv2.COLOR_BGR2YCrCb)
    hr = tf.image.crop_to_bounding_box(hr, 0, 0,IMG_SIZE ,IMG_SIZE)
    # lr = tf.image.resize(hr, [HR_SIZE//SCALE, HR_SIZE//SCALE])
    lr = hr
    lr = tf.cast(lr, tf.uint8)
    return lr

def super_resolve(original_img_path):
    # lr - low resolution image
    # hr - high resolution image (cropped from original)
    # sr - super resolved image from lr
    lr= load_test_image(original_img_path)
    lr_ycrcb = lr.numpy().copy()    
    lr = cv2.cvtColor(lr.numpy(), cv2.COLOR_YCrCb2RGB)
    #hr = cv2.cvtColor(hr.numpy(), cv2.COLOR_YCrCb2RGB)   
    lr_ycrcb = tf.cast(lr_ycrcb, tf.float32)
    lr_y = lr_ycrcb[:, :, 0:1] / 255.
    lr_cr = lr_ycrcb[:, :, 1:2]
    lr_cb = lr_ycrcb[:, :, 2:3]
    lr_y = tf.expand_dims(lr_y, axis=0)
    
    # Use TFLite to super resolve luminance
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    tflite_model = converter.convert()
    interpreter = tf.lite.Interpreter(model_content=tflite_model)
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    interpreter.set_tensor(input_details[0]['index'], lr_y)
    interpreter.invoke()
    sr_y = interpreter.get_tensor(output_details[0]['index'])
    
    open("ESPCN.tflite", 'wb').write(tflite_model)
    
    # Transform super resolved image back into RGB color space
    sr_y = sr_y * 255
    sr_cr = tf.image.resize(lr_cr, [IMG_SIZE*SCALE, IMG_SIZE*SCALE], tf.image.ResizeMethod.BICUBIC)
    sr_cb = tf.image.resize(lr_cb, [IMG_SIZE*SCALE, IMG_SIZE*SCALE], tf.image.ResizeMethod.BICUBIC)
    sr = tf.concat([sr_y[0], sr_cr, sr_cb], axis=2)
    sr = tf.clip_by_value(sr, 0, 255)
    sr = tf.round(sr)
    sr = tf.cast(sr, tf.uint8)
    sr = cv2.cvtColor(sr.numpy(), cv2.COLOR_YCrCb2RGB)
    return sr
    

In [ ]:
## Use model "export_ep700" created from models/ESPCN(ep_700).ipynb
model = keras.models.load_model('export_ep700')

## Object Detection

In [ ]:
# use "classes.names" from repository's data/classes.names
# copy the file under darknet_for_colab/data
!cp "classes.names" "./data/classes.names"

In [ ]:
assert os.getcwd()=='/content/darknet_for_colab', 'Directory should be "/content/darknet_for_colab" instead of "{}"'.format(os.getcwd())

In [ ]:
from ctypes import *
import math
import random
import os
import cv2
import numpy as np
import time
import darknet
import argparse

from IPython.display import clear_output
from google.colab.patches import cv2_imshow
from PIL import Image

import easydict

######## Change Directory ########
args = easydict.EasyDict({
 
        # insert test video path
        "video": 'video.mp4',
 
        # use "yolov4_custom_test.cfg" from repository's data/yolov4_custom_test.cfg
        "config": "yolov4_custom_test.cfg",
 
        # use "yolo4_custom_train.weights" from repository's data/yolov4_custom_train.weights
        "weights": 'yolov4_custom_train.weights',
 
        # use "classes.names" from repository's data/classes.names
        "label": "classes.names",
 
        "meta": "./data/yolov4.data",
 
        "output": "./output.mp4"
 
})
args = vars(args)

def check_argument(args):
    assert os.path.isfile(args["video"]) == True, "Can't find " + args["video"]
    assert os.path.isfile(args["config"]) == True, "Can't find " + args["config"]
    assert os.path.isfile(args["weights"]) == True, "Can't find " + args["weights"]
    assert os.path.isfile(args["meta"]) == True, "Can't find " + args["meta"]
    assert os.path.isfile(args["label"]) == True, "Can't find " + args["label"]

def convertBack(x, y, w, h):
    xmin = int(round(x - (w / 2)))
    xmax = int(round(x + (w / 2)))
    ymin = int(round(y - (h / 2)))
    ymax = int(round(y + (h / 2)))
    return xmin, ymin, xmax, ymax


def cvDrawBoxes(detections, img):
    for detection in detections:
        x, y, w, h = detection[2][0],\
            detection[2][1],\
            detection[2][2],\
            detection[2][3]
        xmin, ymin, xmax, ymax = convertBack(
            float(x), float(y), float(w), float(h))
        pt1 = (xmin, ymin)
        pt2 = (xmax, ymax)
        color = [int(c) for c in COLORS[LABELS.index(detection[0].decode())]]
        #print(color, type(color))
        # if detection[1] * 100 >= 60:
        cv2.rectangle(img, pt1, pt2, color, 1)
        cv2.putText(img,
                  detection[0].decode() +
                  " [" + str(round(detection[1] * 100, 2)) + "]",
                  (pt1[0], pt1[1] - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                  color, 2)
    return img


netMain = None
metaMain = None
altNames = None


def YOLO():


    global metaMain, netMain, altNames, COLORS, LABELS
    
    videoPath = args["video"]
    configPath = args["config"]
    weightPath = args["weights"]
    metaPath = args["meta"]
    labelsPath = args["label"]
    outputPath = args ["output"]
    
    check_argument(args)
    
    LABELS = open(labelsPath).read().strip().split("\n")
    #print(LABELS, len(LABELS))
    np.random.seed(42)
    COLORS = np.random.randint(0, 255, size=(len(LABELS), 3), dtype="uint8")
    
    if not os.path.exists(configPath):
        raise ValueError("Invalid config path `" +
                         os.path.abspath(configPath)+"`")
    if not os.path.exists(weightPath):
        raise ValueError("Invalid weight path `" +
                         os.path.abspath(weightPath)+"`")
    if not os.path.exists(metaPath):
        raise ValueError("Invalid data file path `" +
                         os.path.abspath(metaPath)+"`")
    if netMain is None:
        netMain = darknet.load_net_custom(configPath.encode(
            "ascii"), weightPath.encode("ascii"), 0, 1)  # batch size = 1
    if metaMain is None:
        metaMain = darknet.load_meta(metaPath.encode("ascii"))
    if altNames is None:
        try:
            with open(metaPath) as metaFH:
                metaContents = metaFH.read()
                import re
                match = re.search("names *= *(.*)$", metaContents,
                                  re.IGNORECASE | re.MULTILINE)
                if match:
                    result = match.group(1)
                else:
                    result = None
                try:
                    if os.path.exists(result):
                        with open(result) as namesFH:
                            namesList = namesFH.read().strip().split("\n")
                            altNames = [x.strip() for x in namesList]
                except TypeError:
                    pass
        except Exception:
            pass
    #cap = cv2.VideoCapture(0)
    cap = cv2.VideoCapture(videoPath)
    cap.set(3, 1280)
    cap.set(4, 720)
    fps = cap.get(5)
    print(fps)
    out = cv2.VideoWriter(
        "{}".format(outputPath), cv2.VideoWriter_fourcc(*"DIVX"), 18.0,
        (darknet.network_width(netMain), darknet.network_height(netMain)))
    print("[INFO] Start the YOLO loop...")

    # Create an image we reuse for each detect
    darknet_image = darknet.make_image(darknet.network_width(netMain),
                                    darknet.network_height(netMain),3)

    print("[INFO] Start processing video...")                                    
    while True:
        prev_time = time.time()
        ret, frame_read = cap.read()

        if not ret:
          break
        
        frame_read = super_resolve(frame_read)
        
        frame_rgb = cv2.cvtColor(frame_read, cv2.COLOR_BGR2RGB)
        frame_resized = cv2.resize(frame_rgb,
                                   (darknet.network_width(netMain),
                                    darknet.network_height(netMain)),
                                   interpolation=cv2.INTER_LINEAR)

        darknet.copy_image_from_bytes(darknet_image,frame_resized.tobytes())

        detections = darknet.detect_image(netMain, metaMain, darknet_image, thresh=0.25)
        image = cvDrawBoxes(detections, frame_resized)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        #print(1/(time.time()-prev_time))
        out.write(image)
        #cv2_imshow(image)
        #cv2.imshow('Demo', image)
        cv2.waitKey(3)
    cap.release()
    out.release()
    print('[INFO] Save processed video as "{}"'.format(outputPath))  
                                  
if __name__ == "__main__":
    YOLO()


30.00974342318935
[INFO] Start the YOLO loop...
[INFO] Start processing video...
INFO:tensorflow:Assets written to: /tmp/tmp98onn1vf/assets
INFO:tensorflow:Assets written to: /tmp/tmpjjai_0mu/assets


INFO:tensorflow:Assets written to: /tmp/tmpjjai_0mu/assets


INFO:tensorflow:Assets written to: /tmp/tmp5lduotw6/assets


INFO:tensorflow:Assets written to: /tmp/tmp5lduotw6/assets


INFO:tensorflow:Assets written to: /tmp/tmp2b1p0hm_/assets


...

INFO:tensorflow:Assets written to: /tmp/tmphp1y0x_8/assets


INFO:tensorflow:Assets written to: /tmp/tmphp1y0x_8/assets


INFO:tensorflow:Assets written to: /tmp/tmplto9rsoa/assets


INFO:tensorflow:Assets written to: /tmp/tmplto9rsoa/assets


[INFO] Save processed video as "./output(video19).mp4"
